# Aries Basic Controller Example
## Controllers Api

This can be used to make connections between two aries agents.

A connection invitation must be created by Alice which is then copied to Bob's notebook to accept the invitation request.

Once Bob accepted the invitation, Alice can accept the invitation response from Bob.

This creates a pairwise relationship between Alice and Bob that is end-to-end encrypted and a trust channel between them for exchanging basic messages or verifiable credentials.

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from aries_basic_controller.aries_controller import AriesAgentController

WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)



## Check the agent has an active connection

**Note: An active connection is required, this should have been established on start up through the python script create_connection.py in the setup folder. If not it is possible to run through the did-exchange tutorial to create one between Alice and Bob**

* [Alice](http://localhost:8888/notebooks/did-exchange-inviter.ipynb)
* [Bob](http://localhost:8889/notebooks/did-exchange-invitee.ipynb)

In [4]:
response = await agent_controller.connections.get_connections()
results = response['results']
#print("Results : ", results)
print('\n')
if len(results) > 0:
    connection = response['results'][0]
    #print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("\nActive Connection ID : ", connection_id)
    else:
        print("No active connection found - wait a bit and execute again")
else:
    print("You must create a connection. A connection is being established in the background.")




Active Connection ID :  6f51c520-913c-4780-941c-c4afbdc7caec


### Pairwise Connection Record Parameters (`/connections`)

 * `connection_id`: the unique connection identifier
 * `state`: `init` / `invitation` / `request` / `response` / `active` / `error` / `inactive`
 * `my_did`: the DID this agent is using in the connection
 * `their_did`: the DID the other agent in the connection is using
 * `their_label`: a connection label provided by the other agent
 * `their_role`: a role assigned to the other agent in the connection
 * `inbound_connection_id`: a connection identifier for the related inbound routing connection
 * `initiator`: `self` / `external` / `multiuse`
 * `invitation_key`: a verification key used to identify the source connection invitation
 * `request_id`: the `@id` property from the connection request message
 * `routing_state`: `none` / `request` / `active` / `error`
 * `accept`: `manual` / `auto`
 * `error_msg`: the most recent error message
 * `invitation_mode`: `once` / `multi`
 * `alias`: a local alias for the connection record

### Get Connections List

In [5]:
response = await agent_controller.connections.get_connections()
print(response)

{'results': [{'created_at': '2020-09-01 12:32:48.878318Z', 'invitation_key': '2QWZP9LjTNCq5EXY4cTbGtgfKD7wJAp4KJBvr7BuqD17', 'my_did': '6i6qJNj4chnju8BKtVTDh5', 'updated_at': '2020-09-01 12:33:01.240107Z', 'their_did': 'Xv3tKU9K8jDP2QCuuGaGs', 'initiator': 'external', 'their_label': 'Bob', 'request_id': 'e1582602-b571-4e2a-a6cc-dc088f5729af', 'accept': 'manual', 'routing_state': 'none', 'invitation_mode': 'once', 'state': 'active', 'connection_id': '6f51c520-913c-4780-941c-c4afbdc7caec'}]}


### Get Connection by Connection ID

In [6]:
print(connection_id)
response = await agent_controller.connections.get_connection(connection_id)
print(response)

6f51c520-913c-4780-941c-c4afbdc7caec
{'created_at': '2020-09-01 12:32:48.878318Z', 'invitation_key': '2QWZP9LjTNCq5EXY4cTbGtgfKD7wJAp4KJBvr7BuqD17', 'my_did': '6i6qJNj4chnju8BKtVTDh5', 'updated_at': '2020-09-01 12:33:01.240107Z', 'their_did': 'Xv3tKU9K8jDP2QCuuGaGs', 'initiator': 'external', 'their_label': 'Bob', 'request_id': 'e1582602-b571-4e2a-a6cc-dc088f5729af', 'accept': 'manual', 'routing_state': 'none', 'invitation_mode': 'once', 'state': 'active', 'connection_id': '6f51c520-913c-4780-941c-c4afbdc7caec'}


# TODO - Implement API endpoint for static connections
### Create a new static connection

### Create invitation request that can be passed to Bob

In [7]:
# Create Invitation
invite = await agent_controller.connections.create_invitation()
connection_id = invite["connection_id"]
print("Connection ID", connection_id)
print("Invitation")
print(invite)

Connection ID 0c057db4-d9f2-45f5-a8c1-516e2a1e12d0
Invitation
{'connection_id': '0c057db4-d9f2-45f5-a8c1-516e2a1e12d0', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '717080aa-401d-4edc-9579-dd89a3469b3c', 'recipientKeys': ['BYcjKjcKJV2NSP8oPH1JZUiDsCsiKZqPzWdwtdsPcsyP'], 'serviceEndpoint': 'http://192.168.65.3:8020', 'label': 'Alice'}, 'invitation_url': 'http://192.168.65.3:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNzE3MDgwYWEtNDAxZC00ZWRjLTk1NzktZGQ4OWEzNDY5YjNjIiwgInJlY2lwaWVudEtleXMiOiBbIkJZY2pLamNLSlYyTlNQOG9QSDFKWlVpRHNDc2lLWnFQeldkd3Rkc1Bjc3lQIl0sICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovLzE5Mi4xNjguNjUuMzo4MDIwIiwgImxhYmVsIjogIkFsaWNlIn0='}


### Receive a connection invitation
#### Example of accepting the invitation from the Alice [notebook](localhost:8088) in Bob's [notebook](localhost:8089);
Copy the invitation request above to Bob's [notebook](localhost:8089)

### Accept the invitation, then move to Alice's notebook

In [8]:
# Receive Invitation
response = await agent_controller.connections.accept_connection(invite["invitation"])
# Print out accepted Invite and Alice's connection ID
print("Connection", response)
alice_id = response["connection_id"]

Connection {'created_at': '2020-09-01 12:34:32.780621Z', 'invitation_key': 'BYcjKjcKJV2NSP8oPH1JZUiDsCsiKZqPzWdwtdsPcsyP', 'my_did': 'JHf4eRqEfvdfotJizn5WxT', 'updated_at': '2020-09-01 12:34:32.842258Z', 'initiator': 'external', 'their_label': 'Alice', 'request_id': '343605c1-3e46-4b26-a3ef-d9d65f7508c9', 'accept': 'manual', 'routing_state': 'none', 'invitation_mode': 'once', 'state': 'request', 'connection_id': 'fe5bb963-9c12-4f9e-ad80-bdea40c3bf8a'}


### Accept Request for Connection by Alice

In [9]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(connection_id)
print("ACCEPT REQUEST")
print(connection)
print("state", connection["state"])

True
ACCEPT REQUEST
{'created_at': '2020-09-01 12:33:17.891802Z', 'invitation_key': 'BYcjKjcKJV2NSP8oPH1JZUiDsCsiKZqPzWdwtdsPcsyP', 'my_did': 'D3zLxJ52zjHRnYEtB1Lvyt', 'updated_at': '2020-09-01 12:35:12.627511Z', 'their_did': 'JHf4eRqEfvdfotJizn5WxT', 'initiator': 'self', 'their_label': 'Alice', 'accept': 'manual', 'routing_state': 'none', 'invitation_mode': 'once', 'state': 'response', 'connection_id': '0c057db4-d9f2-45f5-a8c1-516e2a1e12d0'}
state response


### Establish Inbound Connection

### Remove Connection

In [10]:
# Remove a connection
connection = await agent_controller.connections.remove_connection(connection_id)
print(connection)

{}


### End of Tutorial



In [6]:
response = await agent_controller.terminate()
print(response)

None
